# Example use of the Shot class

The shot class features 
* A Dictionary / list of the signals within this shot
* Pre-processing functionality for the data signals within a shot
* Data loading of the signal data (implemented by calling the signal's getter methods)
* a Target function

In [14]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from plasma.primitives.machine import MachineD3D
from plasma.data.user_signals import fs07, dens, pradcore
from plasma.primitives.shots import Shot

import numpy as np

When instantiating a shot it is empty

In [24]:
my_shot = Shot(183463, MachineD3D, [fs07, dens, pradcore])

In [25]:
dummy_conf = {"paths": {"data": "lalala", "signal_prepath": "/projects/FRNN/signal_data_new_2021/"}}

In [ ]:
# We 
my_shot.load_signal_data(dummy_conf)

In [12]:
data.ndim

2

In [13]:
data.size

100